In [1]:
from __future__ import division
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import matplotlib.transforms
from tqdm import tqdm
import pandas as pd
import random
from scipy import integrate
import time
import copy
from functools import partial
import itertools
import pickle
from numpy.random import dirichlet
import numbers

try:
    import cvxpy as cvx
    cvxpy_installed = True
except:
    print('cvxpy not installed. Community.SteadyState() not available.')
    cvxpy_installed = False

In [2]:
colors={};
colors[0] = 'k';
colors[1] = (0,0.6,0.1);
colors[2] = (0.3,0.6,0.1);
colors[3] = (0.8,0.4,0.1);
colors[4] = (0.6,0.1,1.);
colors[5] = (0.45,0.1,1.);
colors[6] = (0.3,0.1,1.);
colors[7] = (0.15,0.1,1.);
colors[8] = (0.0,0.1,1.);
colors[9] = (0.0,0.1,0.85);
colors[10] = (0.,0.1,0.7);
colors[11] = (0.,0.1,0.55);

In [10]:
def save_obj(obj, name ):
    with open('./'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('./' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

# Convex optimization - frontend

The core of Convex optimization used in this code comes from 
**[The Community Simulator: A Python package for microbial ecology](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0230430)**
Marsland R, Cui W, Goldford J, Mehta P (2020) The Community Simulator: A Python package for microbial ecology. PLOS ONE 15(3): e0230430.
[https://doi.org/10.1371/journal.pone.0230430](https://doi.org/10.1371/journal.pone.0230430)

In [5]:
#DEFAULT PARAMETERS FOR CONSUMER AND METABOLIC MATRICES, AND INITIAL STATE
a_default = {'sampling':'Shuffle', #{'Gaussian','Binary','Gamma'} specifies choice of sampling algorithm
          'q': 0.0, #Preference strength of specialist families (0 for generalist and 1 for specialist)
          'tau' : 50, # resource supply rate
          'R0_food':10, #unperturbed fixed point for supplied food
          'Sgen': 0, #Number of generalist species (unbiased sampling over alll resource classes)
          'c1':1., #Specific consumption rate in binary model
          'supply':'external', #resource supply (see dRdt)
          'n_wells':10, #Number of independent wells
          'norm': 0, # growth rate tradeoff as a normalization of c.
             
        ### model-dependent variables
          'regulation':'independent', #metabolic regulation (see dRdt)
          'response':'type I', #functional response (see dRdt)
          'l':0., #Leakage fraction
          'p_cf' : 0.0, # CF probability
             
        ### Species & Resource number dependent variables
          'S':100, #Number of species per well (randomly sampled from the pool of size Stot = sum(SA) + Sgen)
          'SA': 60*np.ones(3), #Number of species in each specialist family (here, 3 families of 60 species)
          'MA': 60*np.ones(3), #Number of resources in each class 
          'muc': 10, #Mean sum of consumption rates (used in all models)
             
        ### irrelevant variables
          'sigc': 0, #Standard deviation of sum of consumption rates for Gaussian and Gamma models
          'c0':0., #Sum of background consumption rates in binary model
          'fs':0., #Fraction of secretion flux with same resource type
          'fw':0., #Fraction of secretion flux to 'waste' resource
          'sparsity':0., #Effective sparsity of metabolic matrix (between 0 and 1)
          'food':0 #index of food source (when a single resource is supplied externally)
         }

In [ ]:
### I'm commenting the options that I do not use. 
def MakeResourceDynamics(assumptions):
    """
    Construct resource dynamics. 'assumptions' must be a dictionary containing at least
    three entries:
    
    response = {'type I', 'type II', 'type III'} specifies nonlinearity of growth law
    
    regulation = {'independent','energy','mass'} allows microbes to adjust uptake
        rates to favor the most abundant accessible resources (measured either by
        energy or mass)
    
    supply = {'off','external','self-renewing'} sets choice of
        intrinsic resource dynamics
        
    Returns a function of N, R, and the model parameters, which itself returns the
        vector of resource rates of change dR/dt
    """
    sigma = {'type I': lambda R,params: params['c']*R,
             #'type II': lambda R,params: params['c']*R/(1+params['c']*R/params['K']),
             'type II': lambda R,params: params['c']*R/(params['cc']+R)
             #'type III': lambda R,params: (params['c']*R)**params['n']/(1+(params['c']*R)**params['n']/params['sigma_max'])
                     }
    
    u = {'independent': lambda x,params: 1.,
         'energy': lambda x,params: (((params['w']*x)**params['nreg']).T
                                      /np.sum((params['w']*x)**params['nreg'],axis=1)).T,
         'mass': lambda x,params: ((x**params['nreg']).T/np.sum(x**params['nreg'],axis=1)).T
        }
    
    h = {'external': lambda R,params: (params['R0']-R)/params['tau'],
    }
    
    J_in = lambda R,params: (u[assumptions['regulation']](params['c']*R,params)
                             *params['w']*sigma[assumptions['response']](R,params))
    J_out = lambda R,params: (params['l']*J_in(R,params)).dot(params['D'].T)
    
    return lambda N,R,params: (h[assumptions['supply']](R,params)
                               -(J_in(R,params)/params['w']).T.dot(N)
                               +(J_out(R,params)/params['w']).T.dot(N))

def MakeConsumerDynamics(assumptions):
    """
    Construct resource dynamics. 'assumptions' must be a dictionary containing at least
    three entries:
    
    response = {'type I', 'type II', 'type III'} specifies nonlinearity of growth law
    
    regulation = {'independent','energy','mass'} allows microbes to adjust uptake
        rates to favor the most abundant accessible resources (measured either by
        energy or mass)
    
    supply = {'off','external','self-renewing','predator'} sets choice of
        intrinsic resource dynamics
        
    Returns a function of N, R, and the model parameters, which itself returns the
        vector of consumer rates of change dN/dt
    """
    sigma = {'type I': lambda R,params: params['c']*R,
             'type II': lambda R,params: params['c']*R/(params['cc']+R)
            }
    
    u = {'independent': lambda x,params: 1.,
         'energy': lambda x,params: (((params['w']*x)**params['nreg']).T
                                      /np.sum((params['w']*x)**params['nreg'],axis=1)).T,
         'mass': lambda x,params: ((x**params['nreg']).T/np.sum(x**params['nreg'],axis=1)).T
        }
    
    J_in = lambda R,params: (u[assumptions['regulation']](params['c']*R,params)
                             *params['w']*sigma[assumptions['response']](R,params))
    J_growth = lambda R,params: (1-params['l'])*J_in(R,params)
    
    return lambda N,R,params: params['g']*N*(np.sum(J_growth(R,params),axis=1)-params['m'])

In [1]:
##define the parameter sampling here
def MakeMatrices(assumptions):
    """
    Construct consumer matrix and metabolic matrix.
    
    assumptions = dictionary of metaparameters
        'sampling' = {'Gaussian','Binary','Gamma'} specifies choice of sampling algorithm
        'SA' = number of species in each family
        'MA' = number of resources of each type
        'Sgen' = number of generalist species
        'muc' = mean sum of consumption rates
        'sigc' = standard deviation for Gaussian sampling of consumer matrix
        'q' = family preference strength (from 0 to 1)
        'c0' = row sum of background consumption rates for Binary sampling
        'c1' = specific consumption rate for Binary sampling
        'fs' = fraction of secretion flux into same resource type
        'fw' = fraction of secretion flux into waste resource type
        'sparsity' = effective sparsity of metabolic matrix (from 0 to 1)
        'wate_type' = index of resource type to designate as "waste"
    
    Returns:
    c = consumer matrix
    D = metabolic matrix
    """
    #PREPARE VARIABLES
    #Force number of species to be an array:
    if isinstance(assumptions['MA'],numbers.Number):
        assumptions['MA'] = [assumptions['MA']]
    if isinstance(assumptions['SA'],numbers.Number):
        assumptions['SA'] = [assumptions['SA']]
    #Force numbers of species to be integers:
    assumptions['MA'] = np.asarray(assumptions['MA'],dtype=int)
    assumptions['SA'] = np.asarray(assumptions['SA'],dtype=int)
    assumptions['Sgen'] = int(assumptions['Sgen'])
    #Default waste type is last type in list:
    if 'waste_type' not in assumptions.keys():
        assumptions['waste_type']=len(assumptions['MA'])-1

    #Extract total numbers of resources, consumers, resource types, and consumer families:
    M = np.sum(assumptions['MA'])
    T = len(assumptions['MA'])
    S = np.sum(assumptions['SA'])+assumptions['Sgen']
    F = len(assumptions['SA'])
    M_waste = assumptions['MA'][assumptions['waste_type']]
    #Construct lists of names of resources, consumers, resource types, and consumer families:
    resource_names = ['R'+str(k) for k in range(M)]
    type_names = ['T'+str(k) for k in range(T)]
    family_names = ['F'+str(k) for k in range(F)]
    consumer_names = ['S'+str(k) for k in range(S)]
    waste_name = type_names[assumptions['waste_type']]
    resource_index = [[type_names[m] for m in range(T) for k in range(assumptions['MA'][m])],
                      resource_names]
    consumer_index = [[family_names[m] for m in range(F) for k in range(assumptions['SA'][m])]
                      +['GEN' for k in range(assumptions['Sgen'])],consumer_names]
    
    #PERFORM GAUSSIAN SAMPLING
    if assumptions['sampling'] == 'Gaussian':
        # sample from shuffled 
        c = pd.DataFrame(np.zeros((S,M)),columns=resource_index,index=consumer_index);
        cc = pd.DataFrame(np.zeros((S,M)),columns=resource_index,index=consumer_index);
        ## new addition: build Kij as c
        # = pd.DataFrame(np.random.normal(1,0.2, size=(S,M)),columns=resource_index,index=consumer_index);
        for k in range(F):
            for j in range(T):
                rs = (np.arange(assumptions['SA'][k]+1)[1:])*2/assumptions['SA'][k];
                dumi1 = np.zeros((assumptions['SA'][k],assumptions['MA'][j]));
                dumi2 = np.zeros((assumptions['SA'][k],assumptions['MA'][j]));
                for j2 in range(assumptions['MA'][j]):
                    dumi1[:,j2] = np.random.normal(1,0.2,size=rs.shape);
                    dumi2[:,j2] = np.random.normal(1,0.2,size=rs.shape);
                c.loc['F'+str(k)]['T'+str(j)] = c.loc['F'+str(k)]['T'+str(j)].values +dumi1*(dumi1>0);
                cc.loc['F'+str(k)]['T'+str(j)] = cc.loc['F'+str(k)]['T'+str(j)].values +dumi2;
                
    #PERFORM Geometric SAMPLING
    elif assumptions['sampling'] == 'Exponential':
        # sample from shuffled 
        c = pd.DataFrame(np.zeros((S,M)),columns=resource_index,index=consumer_index);
        cc = pd.DataFrame(np.zeros((S,M)),columns=resource_index,index=consumer_index);
        ## new addition: build Kij as c
        # = pd.DataFrame(np.random.normal(1,0.2, size=(S,M)),columns=resource_index,index=consumer_index);
        for k in range(F):
            for j in range(T):
                rs = (np.arange(assumptions['SA'][k]+1)[1:])*2/assumptions['SA'][k];
                dumi1 = np.zeros((assumptions['SA'][k],assumptions['MA'][j]));
                dumi2 = np.zeros((assumptions['SA'][k],assumptions['MA'][j]));
                for j2 in range(assumptions['MA'][j]):
                    dumi1[:,j2] = np.random.exponential(1,size=rs.shape);
                    dumi2[:,j2] = np.random.normal(1,0.2,size=rs.shape);
                c.loc['F'+str(k)]['T'+str(j)] = c.loc['F'+str(k)]['T'+str(j)].values +dumi1;
                cc.loc['F'+str(k)]['T'+str(j)] = cc.loc['F'+str(k)]['T'+str(j)].values +dumi2;
                                
    elif assumptions['sampling'] == 'Shuffle':
        # sample from shuffled 
        c = pd.DataFrame(np.zeros((S,M)),columns=resource_index,index=consumer_index);
        cc = pd.DataFrame(np.zeros((S,M)),columns=resource_index,index=consumer_index);
        ## new addition: build Kij as c
        for k in range(F):
            for j in range(T):
                rs = (np.arange(assumptions['SA'][k]+1)[1:])*2/assumptions['SA'][k];
                dumi1 = np.zeros((assumptions['SA'][k],assumptions['MA'][j]));
                dumi2 = np.zeros((assumptions['SA'][k],assumptions['MA'][j]));
                for j2 in range(assumptions['MA'][j]):
                    dumi1[:,j2] = np.random.permutation(rs);
                    dumi2[:,j2] = np.random.normal(1,0.2,size=rs.shape);
                c.loc['F'+str(k)]['T'+str(j)] = c.loc['F'+str(k)]['T'+str(j)].values +dumi1;
                cc.loc['F'+str(k)]['T'+str(j)] = cc.loc['F'+str(k)]['T'+str(j)].values +dumi2;
            
    elif assumptions['sampling'] == 'BinaryShuffle':
        # sample from shuffled 
        c = pd.DataFrame(np.zeros((S,M)),columns=resource_index,index=consumer_index);
        cc = pd.DataFrame(np.zeros((S,M)),columns=resource_index,index=consumer_index);
        ## new addition: build Kij as c
        for k in range(F):
            for j in range(T):
                rs = (np.arange(assumptions['SA'][k]+1)[1:])*2/assumptions['SA'][k];
                dumi1 = np.zeros((assumptions['SA'][k],assumptions['MA'][j]));
                dumi2 = np.zeros((assumptions['SA'][k],assumptions['MA'][j]));
                dumi3 = np.zeros((assumptions['SA'][k],assumptions['MA'][j]));
                for j2 in range(assumptions['MA'][j]):
                    dumi1[:,j2] = np.random.permutation(rs);
                    dumi2[:,j2] = np.random.normal(1,0.2,size=rs.shape);
                    dumi3[:,j2] = np.random.random(size=rs.shape)>assumptions['p'];
                dumi3[np.arange(dumi3.shape[0]), np.random.randint(dumi3.shape[1], size=dumi3.shape[0])]=1;
                c.loc['F'+str(k)]['T'+str(j)] = c.loc['F'+str(k)]['T'+str(j)].values +dumi1*dumi3;
                cc.loc['F'+str(k)]['T'+str(j)] = cc.loc['F'+str(k)]['T'+str(j)].values +dumi2;
           
    else:
        print('Invalid distribution choice. Valid choices are kind=Gaussian, kind=Binary, kind=Gamma, kind=Uniform.')
        return 'Error'
    
    
    #metabolic matrix from binomial distribution
    DTa = np.ones((M,M));
    if(assumptions['p_cf']!=0):
        DTa = np.zeros((M,M));
        for i in np.arange(M-1)+1:
            DTa[i,:i] = np.random.binomial(1, p=assumptions['p_cf'], size=i); #one-way CF
            #DTa[i] = np.random.binomial(1, p=assumptions['p_cf'], size=M)/M; # all-way CF
            DTa[i,i] = 0;
    DT = pd.DataFrame(DTa,columns=resource_index,index=resource_index);  
        
    if(assumptions['norm']==1):
        c = (c.T/np.sum(c.T)).T
    elif(assumptions['norm']==2):
        c = (c.T/np.sum( (c*c).T)**0.5 ).T
        
    return c, cc, DT.T

### Liebig's Model of the Minimum definition

In [1]:
def sigma_LLM(R,params):
    return params['c']*R/(params['cc']+R)
def dNdt_LLM(N,R,params):
    return N*(np.min(sigma_LLM(R,params),axis=1)-params['m'])
def dRdt_LLM(N,R,params):
    return (params['R0']-R)*params['m'] - (params['cc']/params['c']).T.dot(N*np.min(sigma_LLM(R,params),axis=1))

# Assembly rule

## Assembly rule simulation

### Default, with convex optimization

In [ ]:
def BottomUp(init_state, dynamics, params, tol=1e-7, max_iters=1000):
    # init_state is the initial state for the full complex community
    # simulate all the pairs for each well in the initial state.
    # return the species fractions
    
    S = int(np.sum(init_state[0])[0]); nsim = len( np.sum(init_state[0]) );
    R = len( params['R0']);
    init_inds = np.where( init_state[0].T>0 )[1].reshape(nsim,S)
    pairs = list(itertools.combinations(np.arange(S), 2)); npair = len(pairs);
    surv = np.zeros( (nsim, npair+1, S) );
    growth = np.zeros(( nsim, S, R ));
    complex_plate = Community(init_state,dynamics,params,parallel=False);
    complex_plate.SteadyState();
    for k in np.arange(nsim):
        for j in np.arange(S):
            growth[k,j,:] = np.array(params['c'])[ init_inds[k,j ], : ];
            surv[k,-1,j] = np.array(complex_plate.N)[ init_inds[k,j], k ];
        if(np.sum(surv[k,-1])>0): surv[k,-1] = surv[k,-1]/np.sum(surv[k,-1]);
    for i in np.arange(npair):
        pair_state = 0*init_state[0].copy(); np_pair_state = np.array(pair_state);
        pair_inds = init_inds[:,pairs][:,i,:];
        np_pair_state[pair_inds.reshape(2*nsim), np.repeat( np.arange(nsim), 2 )]=1;
        pair_state = np_pair_state;
        pair_plate = Community( (pair_state, init_state[1]),dynamics,params,parallel=False );
        pair_plate.SteadyState();
        for k in np.arange(nsim):
            for j in np.arange(S):
                surv[k,i,j] = np.array(pair_plate.N)[ init_inds[k,j], k ];
            if(np.sum(surv[k,i])>0): surv[k,i] = surv[k,i]/np.sum(surv[k,i]);
    return surv, growth

In [ ]:
def BottomUp_propagate(init_state, dynamics, params):
    # init_state is the initial state for the full complex community
    # simulate all the pairs for each well in the initial state.
    # return the species fractions
    
    S = int(np.sum(init_state[0])[0]); nsim = len( np.sum(init_state[0]) );
    R = len( params['R0']);
    init_inds = np.where( init_state[0].T>0 )[1].reshape(nsim,S)
    pairs = list(itertools.combinations(np.arange(S), 2)); npair = len(pairs);
    surv = np.zeros( (nsim, npair+1, S) );
    growth = np.zeros(( nsim, S, R ));
    complex_plate = Community(init_state,dynamics,params,parallel=False);
    complex_plate.Propagate(T=1e3, compress_species=False);
    for k in np.arange(nsim):
        for j in np.arange(S):
            growth[k,j,:] = np.array(params['c'])[ init_inds[k,j ], : ];
            surv[k,-1,j] = np.array(complex_plate.N)[ init_inds[k,j], k ];
        if(np.sum(surv[k,-1])>0): surv[k,-1] = surv[k,-1]/np.sum(surv[k,-1]);
    for i in np.arange(npair):
        ind1 = pairs[i][0]; ind2 = pairs[i][1];
        pair_state = 0*init_state[0].copy(); np_pair_state = np.array(pair_state);
        pair_inds = init_inds[:,pairs][:,i,:];
        np_pair_state[pair_inds.reshape(2*nsim), np.repeat( np.arange(nsim), 2 )]=1;
        pair_state = np_pair_state;
        pair_plate = Community( (pair_state, init_state[1]),dynamics,params,parallel=False );
        pair_plate.Propagate(T=1e3, compress_species=False);
        for k in np.arange(nsim):
            surv[k,i,ind1] = np.array(pair_plate.N)[ init_inds[k,ind1], k ];
            surv[k,i,ind2] = np.array(pair_plate.N)[ init_inds[k,ind2], k ];
            if(np.sum(surv[k,i])>0): 
                surv[k,i] = surv[k,i]/np.sum(surv[k,i]);
    return surv, growth

### Bistability, without convex optimization

In [ ]:
def BottomUp_b(d, R, RA, c0=[1,1,1], tmax=1000, nstep=10000):
    t = np.linspace(0,tmax,nstep); 
    surv = np.zeros( (10, 3) );
    nc0s = np.zeros((10,3));
    nc0s[0]=[0.99,0.01,0]; nc0s[1]=[0.01,0.99,0];nc0s[2]=[0.01,0.,0.99];nc0s[3]=[0.99,0.,0.01];nc0s[4]=[0.,0.99,0.01];nc0s[5]=[0,0.01,0.99];nc0s[6]=[0.99,0.005,0.005];nc0s[7]=[0.005,0.99,0.005];nc0s[8]=[0.005,0.005,0.99];nc0s[9]=[0.33,0.33,0.33];
    for i in np.arange(10):
        nc0=np.array([nc0s[i,0],nc0s[i,1],nc0s[i,2],c0[0], c0[1], c0[2] ]);
        nc = integrate.odeint(dnc33, nc0, t, args= (d,R,RA,c0) );
        if(np.sum(nc[-1,:3]>0 )):
            surv[i] = nc[-1,:3]/np.sum(nc[-1,:3]);
    return surv

## Assembly rule predictions

### Quantitative predictions

In [20]:
def arule_h( surv, repeat=True, method=0, thres=1e-4 ):
    S = surv.shape[-1]; nsim = surv.shape[0];
    pred = np.zeros((nsim, S));
    fracs = surv / np.sum(surv, axis=-1, keepdims=True);
    for k in np.arange(nsim):
        for i in np.arange(S):
            ind = fracs[k,:-1,i]>0;
            if(method==0):
                pred[k,i] = 1/( np.sum(1/fracs[k,:-1,i][ind]-1) +1);
            elif(method==1):
                pred[k,i] = 1/np.average(1/fracs[k,:-1,i][ind]);
            elif(method==2):
                pred[k,i] = (np.prod(fracs[k,:-1,i][ind]))**(1/len(ind));
            elif(method==3):
                pred[k,i] = np.average(fracs[k,:-1,i][ind]);
            elif(method==4):
                pred[k,i] = (np.prod(fracs[k,:-1,i][ind]))**(1/len(ind));
        if(method==4):
            dumi = np.copy(pred[k]);
            for i in np.arange(S):
                ind = fracs[k,:-1,i]>0;
                pred[k,i] = (np.prod( fracs[k,:-1,i][ind]**dumi[ind] ))**(1/ np.sum(dumi[ind]) );
    pred = pred/np.sum(pred, axis=-1, keepdims=True);
    if(repeat):
        pairs = list(itertools.combinations(np.arange(S),2));
        surv = surv>thres;
        pairs_pool = np.zeros((len(pairs), S));
        for i in np.arange(len(pairs)):
            pairs_pool[i, pairs[i][0]]=1; pairs_pool[i, pairs[i][1]]=1;  
        fracs0 = np.zeros((nsim, S, S)); # pairwise outcome
        for i in np.arange(len(pairs)):
            fracs0[:, pairs[i][0], pairs[i][1] ] = fracs[:,i,pairs[i][0]];
            fracs0[:, pairs[i][1], pairs[i][0] ] = fracs[:,i,pairs[i][1]];
        for k in ( np.arange(nsim)):
            cnt=1; 
            while( cnt>0 ):
                ind1 = np.where(pred[k]<thres)[0];
                ind2 = list( np.where(pred[k]>=thres)[0] );
                cnt=0;
                ind3 = [];
                for i in ind1:
                    if( np.all(fracs0[k,i,ind2]>thres) ):
                        cnt+=1;
                        ind3 = ind3 + [i];
                for i in np.array(ind3):
                    fracs0[k,i,i] = 1;
                    if( np.all(fracs0[k,i,ind3]>thres) ):
                        ind2 = ind2+[i];
                    fracs0[k,i,i] = 0;
                ind2 = np.array(ind2);
                if(len(ind2)>1):
                    for i in ind2:
                        ind = ind2[ind2!=i];
                        if(method==0):
                            pred[k,i] = 1/( np.sum(1/fracs0[k,i,ind]-1) +1);
                        elif(method==1):
                            pred[k,i] = 1/np.average(1/fracs0[k,i,ind]);
                        elif(method==2):
                            pred[k,i] = (np.prod(fracs0[k,i,ind]))**(1/len(ind));
                        elif(method==3):
                            pred[k,i] = np.average(fracs0[k,i,ind])
    pred = pred/np.sum(pred, axis=-1, keepdims=True);
    return pred;

### 3-species predictions with classification

In [2]:
def arule(surv, thres=1e-4):
    surv = surv>thres;
    nsurv = np.sum(surv[:,:3,:], axis=1); nsim = surv.shape[0]; # count pairwise competitions that each species survived
    nsurvf = np.sum( surv[:,-1,:] , axis=1); 
    outcome = np.zeros((7,4)); #all, predicted, expected violation, unexpected violation
    violations = np.zeros(nsim);
    types = np.zeros((6,3)); ##types: classification of pairwise outcomes by counting each species' survival.
    types[0]=[0,1,2]; types[1]=[1,1,1]; types[2]=[1,1,2]; types[3]=[0,2,2]; types[4]=[1,2,2]; types[5]=[2,2,2];
    for i in np.arange(nsim):
        if( nsurvf[i] == 0 ): continue; # bug: extinction of all species
        for j in np.arange(6):
            if( np.all( np.sort(nsurv[i])== types[j] ) ): break;
        if(j==0): # 3 exclusions, one winning twice
            outcome[j,0]+=1; pred=[0,0,0]; pred[np.argmax(nsurv[i])]=1; 
            outcome[j,1]+=np.all(surv[i,-1]==pred); 
            if( np.any(surv[i,-1]!=pred) ): 
#                print(i, pred, surv[i]);
                outcome[j,3]+=1;
        elif(j==1): # cycle
            outcome[j,0]+=1; pred=[0,0,0]; outcome[j,3]+=1;
#            print(i, pred, surv[i]);
        elif(j==2): # one coexistence and two exclusions
            det = np.linalg.det( surv[i,:3] ) # times that each species survived
            
            if(det==0): # one species won in both of two exclusions
                outcome[j+1,0]+=1; pred=[0,0,0]; pred[np.argmax( np.sum(surv[i,:3], axis=0) ) ]=1;
                outcome[j+1,1]+=np.all(surv[i,-1]==pred);
                if( np.any(surv[i,-1]!=pred) ): 
#                    print(i, pred, surv[i]);
                    outcome[j,3]+=1;
            else: # A->B, B->C, C-><-A
                outcome[j,0]+=1; pred = surv[ i, np.argmax( np.sum(surv[i,:3], axis=1) )  ];
                outcome[j,1]+=np.all(surv[i,-1]==pred);
                if( np.any(surv[i,-1]!=pred) ): 
#                    print(i, pred, surv[i]);
                    outcome[j,3]+=1;
                
            #if(np.sum(surv[i,winner])==1):
            #    outcome[j+1,0]+=1; pred = surv[ i, np.argmax( np.sum(surv[i,:3], axis=0) )  ];
            #    outcome[j+1,1]+=np.all(surv[i,-1]==pred);
            #    if( np.any(surv[i,-1]!=pred) ): print(i, pred, surv[i])
            #elif(np.sum(surv[i,winner])==2):
            #    outcome[j,0]+=1; pred=[0,0,0]; #pred[winner]=1; 
            #    pred = surv[ i, np.argmax( np.sum(surv[i,:3], axis=0) )  ];
            #    outcome[j,1]+=np.all(surv[i,-1]==pred); 
            #    if( np.any(surv[i,-1]!=pred) ): print(i, surv[i])
        elif(j==3): #one species lost both exclusions, two winners coexist
            outcome[j+1,0] += 1; pred=[1,1,1]; pred[np.argmin(nsurv[i])]=0; outcome[j+1,1]+=np.all(surv[i,-1]==pred);
            if( np.any(surv[i,-1]!=pred) ): 
#                print(i, pred, surv[i]);
                outcome[j+1,3]+=1;
        elif(j==4): # two coexistence and one exclusion
            outcome[j+1,0] += 1; pred=[1,1,1]; pred[np.argmin(nsurv[i])]=0;
            if( np.all(surv[i,-1]==pred) ):
                outcome[j+1,1]+=1; 
            else:
                pred2= 1 - surv[ i, np.argmin( np.sum(surv[i,:3], axis=1) )  ]; # reverse of the exclusion outcome  
                if( np.all(surv[i,-1]==pred2) ):
                    outcome[j+1,2]+=1; 
                else:
#                    print(i, pred, surv[i]);
                    outcome[j+1,3]+=1;
        elif(j==5):
            outcome[j+1,0] += 1; pred=[1,1,1]; 
            if( np.all(surv[i,-1]==pred) ):
                outcome[j+1,1]+=1; 
            else:
                if( np.any( [np.all(surv[i,-1]==[0,1,1]), np.all(surv[i,-1]==[1,0,1]), np.all(surv[i,-1]==[1,1,0]) ] ) ):
                    outcome[j+1,2]+=1;
                else:
#                    print(i, pred, surv[i]);
                    outcome[j+1,3]+=1;
        else: 
            print(i);
        violations[i]+=j*(1-np.all(surv[i,-1]==pred));
    return outcome, violations;    

### 3-species prediction with classification, with bistable pairs

In [1]:
def arule_b( surv, thres=1e-2):
    S = 3; nsim = surv.shape[0];
    ### first, filter out bistable outcomes.
    surv = 1*(surv>thres);
    checks = np.zeros((nsim,3));
    unobss = np.zeros(nsim);
    ## 4 types of edges -> 3^4 possibilities...? very difficult to do. 
    ## instead, only consider cases where trio outcome is bistable.
    cnt1=0; cnt2=0; cnt3=0; cnt4=0;
    types=[];
    ind_viol = [];
    for k in np.arange(nsim):
        if( np.all(surv[k,0]+surv[k,1]==np.array([1,1,0])) ): checks[k,0]=1; # pair 1 is bistable
        if( np.all(surv[k,2]+surv[k,3]==np.array([1,0,1])) ): checks[k,1]=1; # pair 2 is bistable
        if( np.all(surv[k,4]+surv[k,5]==np.array([0,1,1])) ): checks[k,2]=1; # pair 3 is bistable
        if( 0 in np.sum(surv[k], axis=-1) ):
            cnt4+=1; checks[k]+=4;  continue; #error: no one survived in some competition
        if( np.sum(surv[k,0]+surv[k,1])%2==1 or np.sum(surv[k,2]+surv[k,3])%2==1 or np.sum(surv[k,4]+surv[k,5])%2==1): 
            cnt3+=1; checks[k]+=4; continue; #error: pairwise outcome has a bistable coexistence
        nb = np.sum(checks[k]);
        if(nb==0 or nb>3): continue; # find outcomes with 1-3 bistable pairs and no obvious error
        mxw = np.argmax(np.sum(surv[k,:6],axis=0)); # species with max win
        mnw = np.argmin(np.sum(surv[k,:6],axis=0)); # species with min win
        ns1 = np.sum(surv[k,:6])/2 ## survival sum in all pairs
        pred = [[0,0,0]];
        if(nb==1):
            ind1 = 2-np.where(checks[k]==1)[0]; 
            ### pair that does not involve ind1 is the only bistable one. 
            ### then the outcome possibilities depend on other two edges' coex/exc
            ## exc exc 1, exc exc 2, exc exc 3, exc co 1, exc co 2, co co
            ns2 = np.sum(surv[k,:6,ind1])/2; ## survival count of the non-bistable species
            if( ns1 == 3): # exc exc
                if(ns2==2): # ind1 excludes both others
                    pred[0][mxw]=1; # 
                    types+=[1]; 
                elif(ns2==1): # ind1 is excluded by one species
                    pred[0][mxw]=1;
                    types+=[2];  
                elif(ns2==0): # ind1 is excluded by both species
                    pred[0][(mnw-1)%3]=1;
                    pred+=[[0,0,0]];
                    pred[1][(mnw+1)%3]=1;
                    types+=[3];
            elif( ns1== 4): # exc coex
                coex = surv[k,np.argmax(np.sum(surv[k,:6], axis=-1))];
                if(ns2==2): # ind1 excludes one, coexists with the other
                    pred[0]=list(coex);
                    types+=[4];
                elif(ns2==1): # ind1 is excluded by one, coexists with the other
                    pred[0]=list(coex);
                    pred+=[list(1-coex)];
                    types+=[5];
            elif( ns1 == 5): # ind1 coexists with others
                pred[0][mxw]=1;
                pred[0][(mxw+1)%3]=1;
                pred+=[[0,0,0]];
                pred[1][mxw]=1;
                pred[1][(mxw-1)%3]=1;
                types+=[6];
        elif(nb==2):
            if(ns1==3): # bist bist exc
                pred[0][(mnw+1)%3]=1;
                pred+=[[0,0,0]];
                pred[1][(mnw-1)%3]=1;
                types+=[7];
            elif(ns1==4): # bist bist coex
                pred[0][mnw]=1;
                pred+=[[1,1,1]];
                pred[1][mnw]=0;
                types+=[8];
        elif(nb==3): # bist bist bist
            pred[0]=[1,0,0];
            pred+=[[0,1,0]];
            pred+=[[0,0,1]];
            types+=[9];
    ### now pred is done! need to check whether each surv[6:] falls into one of pred. 
        trio = [];
        for i in np.arange(4):
            trio += [list(surv[k,6+i])];
        viol = [x for x in trio if x not in pred];# + [x for x in pred if x not in trio];
        unobs = [x for x in pred if x not in trio];
        unobss[k] = len(unobs)
        if(len(viol)==0): cnt1+=1;
        else: 
            cnt2+=1;
            ind_viol += [k];
            print(surv[k], mxw,mnw,pred,ns1,nb)
    print(cnt1, cnt2, cnt3, cnt4) # agreement, violation, bistable coex?, no survivor
    return (np.sum(checks, axis=-1), types, ind_viol, unobss)

### Multispecies qualitative prediction (obsolete)

In [ ]:
def arule_m( surv, thres=1e-2 ): #surv = np.zeros((nsim,((S*(S-1))//2)+1,S));
    S = surv.shape[-1]; nsim = surv.shape[0];
    pairs = list(itertools.combinations(np.arange(S),2));
    surv = surv>thres;
    pairs_pool = np.zeros((len(pairs), S));
    for i in np.arange(len(pairs)):
        pairs_pool[i, pairs[i][0]]=1; pairs_pool[i, pairs[i][1]]=1;    
    prediction = np.zeros(nsim);
    for k in np.arange(nsim):
        # two ways.
        # 1. build up prediction by starting from the maximal survivor in pairs and adding species one by one.
        # 2. among survivors in complex community, check if they all pairwisely coexist. among non-survivors, check if they are all excluded.
        # Method 1.
        #pred = np.zeros(S);
        #survcount = np.sum(surv[k,:-1], axis=0); 
        #pred[np.argmax(survcount)]=1;
        #for i in np.arange(S):
        #    inds = np.where(pred == 1)[0];
        #    check=1;
        #    for j in inds:
        #        ind = np.where(pairs_pool[:,i]*pairs_pool[:,j]==1)[0][0];
        #        if( surv[k, ind, i] == 0): check=0; break;
        #    pred[i]=check;
        #prediction[k] =  np.sum((surv[k, -1] - pred)**2)/S ;
        # Method 2.
        pred = np.zeros(S);
        survivor = np.where( surv[k,-1]==1 )[0];
        dead = np.where( surv[k,-1]==0 )[0];
        for i in survivor:
            check=1;
            for j in survivor:
                ind = np.where(pairs_pool[:,i]*pairs_pool[:,j]==1)[0][0];
                if( surv[k, ind, i] == 0): check=0; break;
            pred[i] = check;
        for i in dead:
            check = 1;
            for j in survivor:
                ind = np.where(pairs_pool[:,i]*pairs_pool[:,j]==1)[0][0];
                check = check*surv[k, ind, i];
            pred[i] = check;
        prediction[k] =  np.sum((surv[k, -1] - pred)**2)/S ;
    return 1-prediction


# Convex optimization - backend

In [7]:
def MakeInitialState(assumptions):
    """
    Construct stochastically colonized initial state, at unperturbed resource fixed point.
    
    assumptions = dictionary of metaparameters
        'SA' = number of species in each family
        'MA' = number of resources of each type
        'Sgen' = number of generalist species
        'n_wells' = number of independent wells in the experiment
        'S' = initial number of species per well
        'food' = index of supplied "food" resource
        'R0_food' = unperturbed fixed point for supplied food resource
    
    Returns:
    N0 = initial consumer populations
    R0 = initial resource concentrations
    """

    #PREPARE VARIABLES
    #Force number of species to be an array:
    if isinstance(assumptions['MA'],numbers.Number):
        assumptions['MA'] = [assumptions['MA']]
    if isinstance(assumptions['SA'],numbers.Number):
        assumptions['SA'] = [assumptions['SA']]
    #Force numbers of species to be integers:
    assumptions['MA'] = np.asarray(assumptions['MA'],dtype=int)
    assumptions['SA'] = np.asarray(assumptions['SA'],dtype=int)
    assumptions['Sgen'] = int(assumptions['Sgen'])

    #Extract total numbers of resources, consumers, resource types, and consumer families:
    M = int(np.sum(assumptions['MA']))
    T = len(assumptions['MA'])
    S_tot = int(np.sum(assumptions['SA'])+assumptions['Sgen'])
    F = len(assumptions['SA'])
    #Construct lists of names of resources, consumers, resource types, consumer families and wells:
    resource_names = ['R'+str(k) for k in range(M)]
    type_names = ['T'+str(k) for k in range(T)]
    family_names = ['F'+str(k) for k in range(F)]
    consumer_names = ['S'+str(k) for k in range(S_tot)]
    resource_index = [[type_names[m] for m in range(T) for k in range(assumptions['MA'][m])],
                      resource_names]
    consumer_index = [[family_names[m] for m in range(F) for k in range(assumptions['SA'][m])]
                      +['GEN' for k in range(assumptions['Sgen'])],consumer_names]
    well_names = ['W'+str(k) for k in range(assumptions['n_wells'])]

    R0 = assumptions['R0_food']*np.ones((M,assumptions['n_wells']))
    N0 = np.zeros((S_tot,assumptions['n_wells']))
    
    #if not isinstance(assumptions['food'],int):
    #    assert len(assumptions['food']) == assumptions['n_wells'], 'Length of food vector must equal n_wells.'
    #    food_list = assumptions['food']
    #else:
    #    food_list = np.ones(assumptions['n_wells'],dtype=int)*assumptions['food']

    #if not (isinstance(assumptions['R0_food'],int) or isinstance(assumptions['R0_food'],float)):
    #    assert len(assumptions['R0_food']) == assumptions['n_wells'], 'Length of food vector must equal n_wells.'
    #    R0_food_list = assumptions['R0_food']
    #else:
    #    R0_food_list = np.ones(assumptions['n_wells'],dtype=int)*assumptions['R0_food']

    for k in range(assumptions['n_wells']):
        N0[np.random.choice(S_tot,size=assumptions['S'],replace=False),k]=1.
        #R0[food_list[k],k] = R0_food_list[k]

    N0 = pd.DataFrame(N0,index=consumer_index,columns=well_names)
    R0 = pd.DataFrame(R0,index=resource_index,columns=well_names)

    return N0, R0


def MakeParams(assumptions):
    """
    Makes a dictionary of parameters, using MakeMatrices for the matrices, MakeInitialState
    for the resource supply point, and setting everything else to 1, except l which is zero.
    
    Parameter values can be modified from 1 (or zero for l) by adding their name-value pairs
    to the assumptions dictionary.
    """

    c, cc, D = MakeMatrices(assumptions)
    N0,R0 = MakeInitialState(assumptions)
    #cc = c.copy();
#    cc = cc*0+np.random.normal(1,0.2, size=cc.shape)
    
    if not isinstance(assumptions['food'],int) or not isinstance(assumptions['R0_food'],int):
        params=[{'c':c,
                 'cc':cc,
                'm':1,
                'w':1,
                'D':D,
                'g':1,
                'l':0,
                'R0':R0.values[:,k],
                'tau':1,
                'r':1,
                'sigma_max':1,
                'nreg':10,
                'n':2
                } for k in range(assumptions['n_wells'])]
        for item in ['m','w','g','l','tau','r','sigma_max','n','nreg']:
            if item in assumptions.keys():
                for k in range(assumptions['n_wells']):
                    params[k][item] = assumptions[item]

    else:
        params={'c':c,
                'cc':cc,
                'm':1,
                'w':1,
                'D':D,
                'g':1,
                'l':0,
                'R0':R0.values[:,0],
                'tau':1,
                'r':1,
                'sigma_max':1,
                'nreg':10,
                'n':2
                }
            
        for item in ['m','w','g','l','tau','r','sigma_max','n','nreg']:
            if item in assumptions.keys():
                params[item] = assumptions[item]

    return params



def SimpleDilution(plate, f0 = 1e-3):
    """
    Returns identity matrix of size plate.n_wells, scaled by f0
    """
    f = f0 * np.eye(plate.n_wells)
    return f

def BinaryRandomMatrix(a,b,p):
    """
    Construct binary random matrix.
    
    a, b = matrix dimensions
    
    p = probability that element equals 1 (otherwise 0)
    """
    r = np.random.rand(a,b)
    m = np.zeros((a,b))
    m[r<p] = 1.0
    
    return m

In [8]:
#Parameter dimensions for MicroCRM and Lotka-Volterra
dim_default = {
                'SxM':['c'],
                'MxM':['Di'],#['D','Di'],
                'SxS':['alpha'],
                'S':['m','g','K'],
                'M':['e','w','r','tau','R0']
                }

class Community:
    def __init__(self,init_state,dynamics,params,dimensions=dim_default,scale=10**9,parallel=True):
        """
        Initialize a new "96-well plate" for growing microbial communities.
        
        init_state = [N0,R0] where N0 and R0 are 2-D arrays specifying the 
            initial consumer and resource concentrations, respectively, in each
            of the wells. Each species of consumer and each resource has its
            own row, and each well has its own column. If N0 and R0 are Pandas
            DataFrames, the row and column labels will be preserved throughout
            all subsequent calculations. Otherwise, standard row and column
            labels will be automatically supplied.
        
        dynamics = [dNdt,dRdt] where dNdt(N,R,params) and dRdt(N,R,params) are 
            vectorized functions of the consumer and resource concentrations
            N and R for a single well. params is a Python dictionary containing
            the parameters that required by these functions, and is passed to 
            the new plate instance in the next argument. 
            
        params = {'name':value,...} is a Python dictionary containing names and values
            for all parameters. Parameters that are matrices or vectors (such as the
            consumer preference matrix) should have their dimensions recorded in the
            next argument. This is done automatically for the parameters of the built-
            in Microbial Consumer Resource Model, but must be done by hand for custom
            models.
        dimensions = {'SxM':[name1,name2,...],...} is a dictionary specifying the 
            dimensions of all the parameters. These are used for compressing 
            the parameter arrays when species or resources are extinct. See default
            dictionary above for proper format. Allowed dimensions are SxM, SxS, 
            MxM, M and S, where M is the number of resource types and S is the number
            of consumer species.
            
        scale is a conversion factor specifying the number of individual microbial 
            cells present when N = 1. It is used in the Passage method defined 
            below to perform multinomial sampling, and controls the strength
            of population noise. 
            
        parallel allows for disabeling parallel integration, which is currently not
            supported for Windows machines
        """
        #SAVE INITIAL STATE
        N, R = init_state
        if not isinstance(N, pd.DataFrame):#add labels to consumer state
            if len(np.shape(N)) == 1:
                N = N[:,np.newaxis]
            column_names = ['W'+str(k) for k in range(np.shape(N)[1])]
            species_names = ['S'+str(k) for k in range(np.shape(N)[0])]
            N = pd.DataFrame(N,columns=column_names)
            N.index = species_names
        if not isinstance(R, pd.DataFrame):#add labels to resource state
            if len(np.shape(R)) == 1:
                R = R[:,np.newaxis]
            resource_names = ['R'+str(k) for k in range(np.shape(R)[0])]
            R = pd.DataFrame(R,columns=N.keys())
            R.index = resource_names
        self.N = N.copy()
        self.R = R.copy()
        self.R0 = R.copy() #(for refreshing media on passaging if "refresh_resource" is turned on)
        self.S, self.n_wells = np.shape(self.N)
        self.M = np.shape(self.R)[0]
        
        #SAVE DYNAMICS
        self.dNdt, self.dRdt = dynamics
        
        #SAVE PARAMETERS
        self.params = params.copy()
        if isinstance(self.params,list): #Allow parameter file to be a list
            assert len(self.params) == self.n_wells, 'Length of parameter list must equal n_wells.'
            for k in range(len(self.params)):
                for item in self.params[k]:#strip parameters from DataFrames if necessary
                    if isinstance(self.params[k][item],pd.DataFrame):
                        if (item != 'c' and item != 'cc'):
                            self.params[k][item]=self.params[k][item].values.squeeze()
                        else:
                            self.params[k][item]=self.params[k][item].values
                    elif isinstance(self.params[k][item],list):
                        self.params[k][item]=np.asarray(self.params[k][item])
                    if 'D' not in self.params[k]:#supply dummy values for D and l if D is not specified
                        self.params[k]['D'] = np.ones((self.M,self.M))
                        self.params[k]['l'] = 0
                self.params[k]['S'] = self.S
        else:
            for item in self.params:#strip parameters from DataFrames if necessary
                if isinstance(self.params[item],pd.DataFrame):
                    if (item != 'c' and item != 'cc'):
                        self.params[item]=self.params[item].values.squeeze()
                    else:
                        self.params[item]=self.params[item].values
                elif isinstance(self.params[item],list):
                    self.params[item]=np.asarray(self.params[item])
            if 'D' not in params:#supply dummy values for D and l if D is not specified
                self.params['D'] = np.ones((self.M,self.M))
                self.params['l'] = 0
            self.params['S'] = self.S
        
        #SAVE DIMENSIONS, SCALE AND PARALLEL
        self.dimensions = dimensions
        self.scale = scale
        self.parallel = parallel
            
    def copy(self):
        return copy.deepcopy(self)

    def Reset(self,init_state):
        """
        Reset plate with new initial state, keeping same parameters.
        """
        self.N, self.R = init_state
        
        if not isinstance(self.N, pd.DataFrame):
            column_names = ['D'+str(k) for k in range(np.shape(self.N)[1])]
            species_names = ['S'+str(k) for k in range(np.shape(self.N)[0])]
            self.N = pd.DataFrame(self.N,columns=column_names)
            self.N.index = species_names
            
        if not isinstance(self.R, pd.DataFrame):
            resource_names = ['R'+str(k) for k in range(np.shape(self.R)[0])]
            self.R = pd.DataFrame(self.R,columns=self.N.keys())
            self.R.index = resource_names
            
        self.R0 = self.R.copy()
        self.S, self.n_wells = np.shape(self.N)
        self.M = np.shape(self.R)[0]
        
    def dydt(self,y,t,params,S_comp):
        """
        Combine N and R into a single vector with a single dynamical equation
        
        y = [N1,N2,N3...NS,R1,R2,R3...RM]
        
        t = time
        
        params = params to pass to dNdt,dRdt
        
        S_comp = number of species in compressed consumer vector
            (with extinct species removed)
        """
        return np.hstack([self.dNdt(y[:S_comp],y[S_comp:],params),
                          self.dRdt(y[:S_comp],y[S_comp:],params)])
    
    def SteadyState(self,tol=1e-7,shift_size=1,alpha=0.5,
                    eps=1e-10,R0t_0=10,max_iters=1000,verbose=False,plot=False):
        """
        Find the steady state using convex optimization.
        
        supply = {external, self-renewing}
        """
        
        #CONSTRUCT FULL SYSTEM STATE
        y_in = self.N.append(self.R).values
        
        #PACKAGE SYSTEM STATE AND PARAMETERS IN LIST OF DICTIONARIES
        if not isinstance(self.params,list):
            params = [self.params]*self.n_wells
        else:
            params = self.params
        well_info = [{'y0':y_in[:,k],'params':params[k]} for k in range(self.n_wells)]
        
        #PREPARE OPTIMIZER FOR PARALLEL PROCESSING
        OptimizeTheseWells = partial(OptimizeWell,tol=tol,alpha=alpha,
                                     shift_size=shift_size,max_iters=max_iters,
                                     eps=eps,R0t_0=R0t_0,verbose=verbose,dimensions=self.dimensions)
        
        
        #IF PARALLEL IS DEACTIVATED, USE ORDINARY MAP
        y_out = np.asarray(list(map(OptimizeTheseWells,well_info))).squeeze().T
        if len(np.shape(y_out)) == 1:#handle case of single-well plate
            y_out = y_out[:,np.newaxis]
        
        #UPDATE STATE VARIABLES WITH RESULTS OF OPTIMIZATION
        self.N = pd.DataFrame(y_out[:self.S,:],
                              index = self.N.index, columns = self.N.keys())
        self.R = pd.DataFrame(y_out[self.S:,:],
                              index = self.R.index, columns = self.R.keys())

        #PRINT DIAGNOSTICS
        dNdt_f = np.asarray(list(map(self.dNdt,self.N.T.values,self.R.T.values,params)))
        dRdt_f = np.asarray(list(map(self.dRdt,self.N.T.values,self.R.T.values,params)))
        
        if plot:
            dNdt_f = np.asarray(list(map(self.dNdt,self.N.T.values,self.R.T.values,params))).reshape(-1)
            dRdt_f = np.asarray(list(map(self.dRdt,self.N.T.values,self.R.T.values,params))).reshape(-1)
            N = self.N.values.reshape(-1)
            R = self.R.values.reshape(-1)
    
            fig,ax = plt.subplots()
            ax.plot(dNdt_f[N>0]/N[N>0],'o',markersize=1)
            ax.set_ylabel('Per-Capita Growth Rate')
            ax.set_title('Consumers')
            plt.show()
            
            fig,ax = plt.subplots()
            ax.plot(dRdt_f/R,'o',markersize=1)
            ax.set_ylabel('Per-Capita Growth Rate')
            ax.set_title('Resources')
            plt.show()
            
    def Propagate(self,T,compress_resources=False,compress_species=True):
        """
        Propagate the state variables forward in time according to dNdt, dRdt.

        T = time interval for propagation

        compress_resources specifies whether zero-abundance resources should be
            ignored during the propagation. This makes sense when the resources
            are non-renewable.

        compress_species specifies whether zero-abundance species should be
            ignored during the propagation. This always makes sense for the
            models we consider. But for user-defined models with new parameter
            names, it must be turned off, since the package does not know how
            to compress the parameter matrices properly.
        """
        #CONSTRUCT FULL SYSTEM STATE
        y_in = self.N.append(self.R).values

        #PACKAGE SYSTEM STATE AND PARAMETERS IN LIST OF DICTIONARIES
        if isinstance(self.params,list):
            well_info = [{'y0':y_in[:,k],'params':self.params[k]} for k in range(self.n_wells)]
        else:
            well_info = [{'y0':y_in[:,k],'params':self.params} for k in range(self.n_wells)]

        #PREPARE INTEGRATOR FOR PARALLEL PROCESSING
        IntegrateTheseWells = partial(IntegrateWell,self,T=T,compress_resources=compress_resources,
                                      compress_species=compress_species)

        #INITIALIZE PARALLEL POOL AND SEND EACH WELL TO ITS OWN WORKER
        if self.parallel:
            pool = Pool()
            y_out = np.asarray(pool.map(IntegrateTheseWells,well_info)).squeeze().T
            pool.close()
        else:
            y_out = np.asarray(list(map(IntegrateTheseWells,well_info))).squeeze().T

        #HANDLE CASE OF A SINGLE-WELL PLATE
        if len(np.shape(y_out)) == 1:
            y_out = y_out[:,np.newaxis]

        #UPDATE STATE VARIABLES WITH RESULTS OF INTEGRATION
        self.N = pd.DataFrame(y_out[:self.S,:],
                              index = self.N.index, columns = self.N.keys())
        self.R = pd.DataFrame(y_out[self.S:,:],
                              index = self.R.index, columns = self.R.keys())

In [9]:
def CompressParams(not_extinct_consumers,not_extinct_resources,params,dimensions,S,M):
    params_comp = params.copy()
    if 'SxM' in dimensions.keys():
        for item in dimensions['SxM']:
            if item in params_comp.keys():
                assert np.shape(params_comp[item])==(S,M), 'Invalid shape for ' + item + '. Please update dimensions dictionary with correct dimensions.'
                params_comp[item]=params_comp[item][not_extinct_consumers,:]
                params_comp[item]=params_comp[item][:,not_extinct_resources]
    if 'MxM' in dimensions.keys():
        for item in dimensions['MxM']:
            if item in params_comp.keys():
                assert np.shape(params_comp[item])==(M,M), 'Invalid shape for ' + item + '. Please update dimensions dictionary with correct dimensions.'
                params_comp[item]=params_comp[item][not_extinct_resources,:]
                params_comp[item]=params_comp[item][:,not_extinct_resources]
    if 'SxS' in dimensions.keys():
        for item in dimensions['SxS']:
            if item in params_comp.keys():
                assert np.shape(params_comp[item])==(S,S), 'Invalid shape for ' + item + '. Please update dimensions dictionary with correct dimensions.'
                params_comp[item]=params_comp[item][not_extinct_consumers,:]
                params_comp[item]=params_comp[item][:, not_extinct_consumers]
    if 'S' in dimensions.keys():
        for item in dimensions['S']:
            if item in params_comp.keys():
                if type(params_comp[item]) == np.ndarray:
                    assert len(params_comp[item])==S, 'Invalid length for ' + item + '. Please update dimensions dictionary with correct dimensions.'
                    params_comp[item]=params_comp[item][not_extinct_consumers]
    if 'M' in dimensions.keys():
        for item in dimensions['M']:
            if item in params_comp.keys():
                if type(params_comp[item]) == np.ndarray:
                    assert len(params_comp[item])==M, 'Invalid length for ' + item + '. Please update dimensions dictionary with correct dimensions.'
                    params_comp[item]=params_comp[item][not_extinct_resources]  
    return params_comp

In [10]:
def OptimizeWell(well_info,supply='external',tol=1e-7,shift_size=1,eps=1e-20,
                 alpha=0.5,R0t_0=10,verbose=False,max_iters=1000,dimensions={}):
    """
    Uses convex optimization to find the steady state of the ecological dynamics.
    """

    assert cvxpy_installed, 'CVXPY not found. To use SteadyState(), please download and install CVXPY from www.cvxpy.org.'
    
    #UNPACK INPUT
    y0 = well_info['y0'].copy()
    S = well_info['params']['S']
    M = len(y0)-S
    N = y0[:S]
    R = y0[S:]
    
    #COMPRESS PARAMETERS TO GET RID OF EXTINCT SPECIES
    assert np.isnan(N).sum() == 0, 'All species must have numeric abundances (not NaN).'
    assert np.isnan(R).sum() == 0, 'All resources must have numeric abundances (not NaN).'
    not_extinct_consumers = N>0
    not_extinct_resources = np.ones(len(R),dtype=bool)
    #Compress parameters
    params_comp = CompressParams(not_extinct_consumers,not_extinct_resources,well_info['params'],dimensions,S,M)    
    S = len(params_comp['c'])
    M = len(params_comp['c'].T)

    
    failed = 0
    if np.max(params_comp['l']) != 0:
        assert supply == 'external', 'Replenishment must be external for crossfeeding dynamics.'
        
        #Make Q matrix and effective weight vector
        w_mat = np.kron(np.ones((M,1)),np.ones((1,M))*params_comp['w'])
        Q = np.eye(M) - params_comp['l']*params_comp['D']*w_mat/(w_mat.T)
        Qinv = np.linalg.inv(Q)
        Qinv_aa = np.diag(Qinv)
        w = Qinv_aa*(1-params_comp['l'])*params_comp['w']/params_comp['tau']
        Qinv = Qinv - np.diag(Qinv_aa)
        
        #Construct variables for optimizer
        G = params_comp['c']*params_comp['w']*(1-params_comp['l'])/w #Divide by w, because we will multiply by wR
        if isinstance(params_comp['m'],np.ndarray):
            h = params_comp['m'].reshape((S,1))
        else:
            h = np.ones((S,1))*params_comp['m']
        
        #Initialize effective resource concentrations
        if len(np.shape(R0t_0)) == 0:
            R0t = R0t_0*np.ones(M)
        else:
            R0t = R0t_0
        
        #Set up the loop
        Rf = np.inf
        Rf_old = 0

        k=0
        ncyc=0
        Delta = 1
        Delta_old = 1
        while np.linalg.norm(Rf_old - Rf) > tol and k < max_iters:
            try:
                start_time = time.time()
        
                wR = cvx.Variable(shape=(M,1)) #weighted resources
        
                #Need to multiply by w to get properly weighted KL divergence
                R0t = np.sqrt(R0t**2+eps)
                wR0 = (R0t*w).reshape((M,1))

                #Solve
                obj = cvx.Minimize(cvx.sum(cvx.kl_div(wR0, wR)))
                constraints = [G@wR <= h, wR >= 0]
                prob = cvx.Problem(obj, constraints)
                prob.solver_stats
                prob.solve(solver=cvx.ECOS,abstol=1e-8,reltol=1e-8,warm_start=True,verbose=False,max_iters=5000)

                #Record the results
                Rf_old = Rf
                Nf=constraints[0].dual_value[0:S].reshape(S)
                Rf=wR.value.reshape(M)/w

                #Update the effective resource concentrations
                R0t_new = params_comp['R0'] + Qinv.dot((params_comp['R0']-Rf)/params_comp['tau'])*(params_comp['tau']/Qinv_aa)
                Delta_R0t = R0t_new-R0t
                R0t = R0t + alpha*Delta_R0t
                
                Delta_old = Delta
                Delta = np.linalg.norm(Rf_old - Rf)
                if verbose:
                    print('Iteration: '+str(k))
                    print('Delta: '+str(Delta))
                    print('---------------- '+str(time.time()-start_time)[:4]+' s ----------------')
            except:
                #If optimization fails, try new R0t
                shift = shift_size*np.random.randn(M)
                R0t = np.abs(R0t + shift)
                
                if verbose:
                    print('Added '+str(shift_size)+' times random numbers')
            k+=1
            #Check for limit cycle
            if np.isfinite(Delta) and Delta > tol and np.abs(Delta-Delta_old) < 0.1*tol:
                ncyc+=1
            if ncyc > 10:
                print('Limit cycle detected')
                k = max_iters

        if k == max_iters:
            print('Maximum iterations exceeded. Try decreasing alpha to improve convergence.')
            failed = 1
        else:
            if verbose:
                print('success')
                
    elif params_comp['l'] == 0:

        G = params_comp['c']*params_comp['tau'] #Multiply by tau, because wR has tau in the denominator
        if isinstance(params_comp['m'],np.ndarray):
            h = params_comp['m'].reshape((S,1))
        else:
            h = np.ones((S,1))*params_comp['m']

        wR = cvx.Variable(shape=(M,1)) #weighted resources

        #Need to multiply by w to get properly weighted KL divergence
        wR0 = (params_comp['R0']*params_comp['w']*np.ones(M)/params_comp['tau']).reshape((M,1))

        #Solve
        obj = cvx.Minimize(cvx.sum(cvx.kl_div(wR0, wR)))
        constraints = [G@wR <= h]
        prob = cvx.Problem(obj, constraints)
        prob.solver_stats
        try:
            prob.solve(solver=cvx.ECOS,abstol=1e-8,reltol=1e-8,warm_start=True,verbose=False,max_iters=50000)
        except:
            failed=1;

    #Record the results

    if not failed:
        Nf=constraints[0].dual_value[0:S].reshape(S)
        Rf=wR.value.reshape(M)*params_comp['tau']/params_comp['w']
        N_new = np.zeros(len(N))
        R_new = np.zeros(len(R))
        N_new[np.where(not_extinct_consumers)[0]] = Nf
        R_new[np.where(not_extinct_resources)[0]] = Rf
    else:
        N_new = np.zeros(len(N))#np.nan*N
        R_new = np.zeros(len(R))#np.nan*R
        if verbose:
            print('Optimization Failed.')
            
    return np.hstack((N_new, R_new))
    

In [11]:
def IntegrateWell(CommunityInstance,well_info,T0=0,T=1,ns=2,return_all=False,log_time=False,
                  compress_resources=False,compress_species=True):
    """
        Integrator for Propagate and TestWell methods of the Community class
        """
    #MAKE LOGARITHMIC TIME AXIS FOR LONG SINGLE RUNS
    if log_time:
        t = 10**(np.linspace(np.log10(T0),np.log10(T0+T),ns))
    else:
        t = np.linspace(T0,T0+T,ns)
    
    #UNPACK INPUT
    y0 = well_info['y0']
    
    #COMPRESS STATE AND PARAMETERS TO GET RID OF EXTINCT SPECIES
    S = well_info['params']['S']
    M = len(y0)-S
    not_extinct = y0>0
    if not compress_species:
        not_extinct[:S] = True
    if not compress_resources:  #only compress resources if we're running non-renewable dynamics
        not_extinct[S:] = True
    S_comp = np.sum(not_extinct[:S]) #record the new point dividing species from resources
    not_extinct_idx = np.where(not_extinct)[0]
    y0_comp = y0[not_extinct]
    not_extinct_consumers = not_extinct[:S]
    not_extinct_resources = not_extinct[S:]

    #Compress parameters
    params_comp = CompressParams(not_extinct_consumers,not_extinct_resources,well_info['params'],CommunityInstance.dimensions,S,M)

    #INTEGRATE AND RESTORE STATE VECTOR TO ORIGINAL SIZE
    if return_all:
        out = integrate.odeint(CommunityInstance.dydt,y0_comp,t,args=(params_comp,S_comp),mxstep=10000,atol=1e-4)
        traj = np.zeros((np.shape(out)[0],S+M))
        traj[:,not_extinct_idx] = out
        return t, traj
    else:
        out = integrate.odeint(CommunityInstance.dydt,y0_comp,t,args=(params_comp,S_comp),mxstep=10000,atol=1e-4)[-1]
        yf = np.zeros(len(y0))
        yf[not_extinct_idx] = out
        return yf

In [ ]:
def TestWell(self,T = 4,well_name = None,f0 = 1.,ns=100,log_time = False,T0=0,
                 compress_resources=False,compress_species=False,show_plots=True,axs=[]):
        """
        Run a single well and plot the trajectory.
        
        T = duration of trajectory
        
        well_name = label of well to run (will choose first well if "None")
        
        f0 = fraction by which to reduce initial consumer populations. This is
            useful when running a serial transfer simulation, where the initial
            populations for the next plate will be a small fraction of the current
            values
            
        ns = number of time points to sample
        
        log_time allows one to use a logarithmic time axis, which is helpful if
            the community has very fast initial transient dynamics followed by 
            a slow convergence to steady state
            
        compress_resources specifies whether zero-abundance resources should be
            ignored during the propagation. This makes sense when the resources
            are non-renewable.
        """
        #EXTRACT STATE OF A SINGLE WELL
        if well_name == None:
            well_name = self.N.keys()[0]
        N_well = self.N.copy()[well_name] * f0
        R_well = self.R.copy()[well_name]
        if isinstance(self.params,list):
            params_well = self.params[np.where(np.asarray(self.N.keys())==well_name)[0][0]]
        else:
            params_well = self.params
        
        #INTEGRATE WELL
        t, out = IntegrateWell(self,{'y0':N_well.append(R_well).values,'params':params_well},T=T,ns=ns,T0=T0,
                               return_all=True,log_time=log_time,compress_resources=compress_resources,
                               compress_species=compress_species)
        
        Ntraj = out[:,:self.S]
        Rtraj = out[:,self.S:]
        
        #PLOT TRAJECTORY
        if show_plots:
            if axs == []:
                fig, axs = plt.subplots(2,sharex=True)
            else:
                assert len(axs) == 2, 'Must supply two sets of axes.'

            if log_time:
                axs[0].semilogx(t,Ntraj)
                axs[1].semilogx(t,Rtraj)
            else:
                axs[0].plot(t,Ntraj)
                axs[1].plot(t,Rtraj)
            axs[0].set_ylabel('Consumer Abundance')
            axs[1].set_ylabel('Resource Abundance')
            axs[1].set_xlabel('Time')

        return t, Ntraj, Rtraj

# Miscellaneous (plotting etc)

In [3]:
def plot_outcome(outcome, figsize=(5.55,2)):
    fig, ax = plt.subplots(figsize=figsize);
    outcome2 = np.copy(outcome);
    # 0123456 -> 034256
    outcome2[1] = outcome[3]; outcome2[2]=outcome[4]; outcome2[3]=outcome[2];
    outcome2[4] = outcome[5]; outcome2[5]=outcome[6];
    #outcome2[1:-1] = outcome[2:];
    ax.bar( np.arange(6), outcome2[:-1,1], color='#7AC943', alpha=0.8 );
    ax.bar( np.arange(6), outcome2[:-1,0]-outcome2[:-1,1], bottom=outcome2[:-1,1], color='#93278F', alpha=0.8);
    labels=[];
    for i in np.arange(6):
        labels+= ['%i / %i' %(outcome2[i,1],outcome2[i,0])];
    ax.set_ylim(0,150); ax.set_xticks([]);
    ax.set_xlim(-0.5,5.5);
    #ax.set_xlabel('3-species pairwise outcomes');
    ax.set_ylabel('Frequency');
    #ax.legend(['Assembly Rule prediction','Assembly Rule violation'], loc='upper right');
    ax.set_yticks([0,100,200])
    #ax.set_xticks(np.arange(6)); #ax.set_yticks();
    #ax.set_xticklabels(labels)
    return fig, ax

In [19]:
def dnc33(nc, t, d, R, RA, c0):
    n1,n2,n3,c1,c2,c3=nc;
    return [ n1*(R[0,0]*c1+R[0,1]*c2+R[0,2]*c3 - d ), n2*(R[1,0]*c1+R[1,1]*c2+R[1,2]*c3 - d ), n3*(R[2,0]*c1+R[2,1]*c2+R[2,2]*c3 - d ), -c1*(RA[0,0]*n1+RA[1,0]*n2+RA[2,0]*n3+d)+d*c0[1],  -c2*(RA[0,1]*n1+RA[1,1]*n2+RA[2,1]*n3+d)+d*c0[1],  -c3*(RA[0,2]*n1+RA[1,2]*n2+RA[2,2]*n3+d)+d*c0[2]  ]
def dnc2(nc, t, d, R, RA,c10,c20):
    n1,n2,c1,c2=nc;
    return [ n1*(R[0,0]*c1+R[0,1]*c2 - d ), n2*(R[1,0]*c1+R[1,1]*c2 - d ), -c1*(RA[0,0]*n1+RA[1,0]*n2+d)+d*c10,  -c2*(RA[0,1]*n1+RA[1,1]*n2+d)+d*c20  ]
def dnc3(nc, t, d, R, RA,c10,c20):
    n1,n2,n3,c1,c2=nc;
    return [ n1*(R[0,0]*c1+R[0,1]*c2 - d ), n2*(R[1,0]*c1+R[1,1]*c2 - d ), n3*(R[2,0]*c1+R[2,1]*c2 - d ), -c1*(RA[0,0]*n1+RA[1,0]*n2+RA[2,0]*n3+d)+d*c10,  -c2*(RA[0,1]*n1+RA[1,1]*n2+RA[2,1]*n3+d)+d*c20  ]

def dnc(nc, t, d, R, RA, c0):
    ns, nr = R.shape;
    n = nc[:ns]; c = nc[ns:];
    dn = n*( np.sum(R*r, axis=1) - d);
    dc = -c*( np.sum(RA.T*n, axis=1) ) + d*(c0-c);
    return np.concatenate([dn,dc]);

def trajc12(n10,n20,c10,c20,d,R, RA, tmax=1000, nstep=10000):
    t = np.linspace(0,tmax,nstep); dt=t[1]-t[0];
    nc0=np.array([n10,n20,c10,c20]);
    nc = integrate.odeint(dnc2, nc0, t, args= (d,R,RA,c10,c20) );
    f = nc[-1,0]/(nc[-1,0]+nc[-1,1]); c=(f,1-f,0);
    return nc,c
def trajc123(n10,n20,n30,c10,c20,d,R, RA, tmax=1000, nstep=10000):
    t = np.linspace(0,tmax,nstep); dt=t[1]-t[0];
    nc0=np.array([n10,n20,n30,c10,c20]);
    nc = integrate.odeint(dnc3, nc0, t, args= (d,R,RA,c10,c20) );
    f1 = nc[-1,0]/(nc[-1,0]+nc[-1,1]+nc[-1,2]);f2 = nc[-1,1]/(nc[-1,0]+nc[-1,1]+nc[-1,2]);f3 = nc[-1,2]/(nc[-1,0]+nc[-1,1]+nc[-1,2]);
    if(f1<1/nstep): f1=0;
    if(f2<1/nstep): f2=0;
    if(f3<1/nstep): f3=0;
    if(f1>1): f1=1;
    if(f2>1): f2=1;
    if(f3>1): f3=1;
    c=(f1,f2,f3);
    return nc,c

In [12]:
def fast(growth):
    # growth: nsim x S x R
    # return order of growth rates (i.e. [0,1,2])
    # np.argsort (sum of growth rates on all resources for each species)
    # then flip to get the maximum in the first
    nsim, S, R = growth.shape;
    return np.argsort( np.sum(growth, axis=-1), axis=-1 )
def pred_growth(surv, growth, thres=1e-2):
    surv = surv>thres;
    order = fast(growth);
    winner = order[:,-1];
    nsim, S = order.shape;
    pred = np.zeros((nsim, S));
    pred[ np.arange(nsim), winner ] = 1;
    prediction = np.zeros(nsim);
    prediction = np.sum( ((surv[:,-1] - pred)**2)/S, axis=-1 );
    return 1-prediction    

In [13]:
def filter_surv(surv, thres=2e-5):
    # filter out extinction cases
    nsim, dumi, S = surv.shape;
    cnt = np.ones(nsim);
    for i in np.arange(nsim):
        for j in np.arange(dumi):
            if(np.sum(surv[i,j] > thres) == 0):
                cnt[i] = 0;
    return cnt==1;